In [1]:
from datetime import datetime, timedelta, date, time
import sys
import os
sys.path.append("/home/sergey/drclinics/common")
sys.path.append("/home/sergey/drclinics/reports")

#sys.path.append("./../common/") 

import numpy as np
import pandas as pd

from universal_connection import UniversalConnection, DBType

In [2]:
from auto_width8 import auto_columns_width

In [3]:
full_df = pd.DataFrame()

start = '2020-06-'

t_start = ''
t_end = ''

for i in range(1, 31):
    
    if i == 30:
        t_start = '2020-06-30'
        t_end = '2020-07-01'
    elif len(str(i)) == 1:    
        t_start = start + '0' + str(i)
        t_end = start + '0' + str(i + 1)
    else:
        t_start = start + str(i)
        t_end = start + str(i + 1)
    
    print(t_start, t_end)
    print('************')
    
    sql = """
    with temp as (
    select 	a.id app_id,
        --
        a.planned, --at time zone 'UTC' planned,
        a.updated, --at time zone 'UTC' updated,
        a.started, --at time zone 'UTC' started,
        a.finished, --at time zone 'UTC' finished,
        a.finished - a.started duration,
        a.started - a.planned waiting,
        --
        a."source",
        a.state,
        --
        concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
        p_patient.birthday patient_birthday,
        p_patient.phone,
        p_patient.email,
        p_patient.sex,
        --
        po.policy_number,
        --
        case 	
            when a.state = 'ACTIVE' or a.state = 'FINISHED'
            then payment.external_id 
            else null
        end transaction_,
        case 	
            when a.state = 'ACTIVE' or a.state = 'FINISHED'
            then invoice.created 
            else null
        end invoice_created,
        case 	
            when a.state = 'ACTIVE' or a.state = 'FINISHED'
            then invoice.payment_system 
            else null
        end payment_system,
        case 	
            when a.state = 'ACTIVE' or a.state = 'FINISHED'
            then invoice_item.price
            else null
        end i_price,
        case 	
            when a.state = 'ACTIVE' or a.state = 'FINISHED'
            then invoice_item.description
            else null
        end invoice_item_description,
        --invoice_item.description,
        --
        concat_ws(' ', p_doctor.last_name, p_doctor.first_name, p_doctor.middle_name) doctor_fio,
        case
        when a."source"='SCHEDULED' and specialty_id is not null then (
            select trim(name)
            from specialty s
            where s.id = a.specialty_id
        )
        else (
            select trim(s.name)
            from specialty s
            inner join doctor_specialty ds
               on ds.specialty_id = s.id
            inner join doctor_specialty_treatment dst
               on dst.doctor_specialty_id = ds.id
            where dst.treatment_id = a.treatment_id
            limit 1 -- из-за одной криво заведенной записи.
        ) end as specialty,
        p3.full_name doctor_provider,
        par_doc."name" doctor_partner,
        --
        a.report_comment,
        --
        review.rating,
        review."comment",
        --
        a.price,
        --
        promo.description,
        --promo.description_tsvector, NE NUJNO
        promotion_code.code_norm,
        partner."name" patient_partner,
        --
        'CHAT' in (
            select reference.code
            from appointment_connection_type
            left join reference on reference.id = appointment_connection_type.reference_id
            where a.id = appointment_connection_type.appointment_id
            ) as is_chat,
        'VIDEO' in (
            select reference.code
            from appointment_connection_type
            left join reference on reference.id = appointment_connection_type.reference_id
            where a.id = appointment_connection_type.appointment_id
            ) as is_video,
        'AUDIO' in (
            select reference.code
            from appointment_connection_type
            left join reference on reference.id = appointment_connection_type.reference_id
            where a.id = appointment_connection_type.appointment_id
            ) as is_audio,
        'PHONE' in (
            select reference.code
            from appointment_connection_type
            left join reference on reference.id = appointment_connection_type.reference_id
            where a.id = appointment_connection_type.appointment_id
            ) as is_phone,
            a.good,
            --
            case when a."source" = 'ACTIVE'
                 then concat_ws(' ', p_doctor.last_name, p_doctor.first_name, p_doctor.middle_name)
                 else null
            end initiator
        --
    from appointment a 
    --
    left join patient on a.patient_id = patient.id 
    left join person p_patient on patient.person_id = p_patient.id 
    --
    left join doctor on a.doctor_id = doctor.id 
    left join person p_doctor on doctor.person_id = p_doctor.id 
    left join provider p3 on p_doctor.provider_id = p3.id
    --
    --left join promotion_doctor pd on pd.doctor_id = doctor.id 
    --left join promotion promo_d on pd.promotion_id = promo_d.id 
    --left join partner par_doc on promo_d.partner_id = par_doc.id
    --left join product_doctor pd on pd.doctor_id = doctor.id 
    --left join product_condition pc_doc on pd.product_condition_id = pc_doc.id 
    --left join promotion promo_d on promo_d.product_condition_id = pc_doc.id
    --left join partner par_doc on promo_d.partner_id = par_doc.id
    left join partner par_doc on p_doctor.partner_id = par_doc.id
    --
    left join review on review.appointment_id = a.id
    --
    left join promotion promo on a.promotion_id = promo.id
    left join promotion_code on promo.promotion_code_id = promotion_code.id
    left join partner on promo.partner_id = partner.id 
    --
    left join product_condition pc 
    on promo.product_condition_id = pc.id
    --
    left join invoice on a.invoice_id = invoice.id
    left join invoice_item on invoice_item.invoice_id = invoice.id
    --
    left join invoice_payments on invoice_payments.invoice_id = invoice.id
    left join payment on payment.id = invoice_payments.payment_id 
    --
    left join product_order_promotion pop on promo.id = pop.promotion_id 
    --
    left join product_order po
    on a.patient_id = po.patient_id
    --and po.promotion_code_id  = promotion_code.id
    --and po.product_id = pc.product_id
    and pop.product_order_id = po.id
    --and po.invoice_id = invoice.id
    --
    where a.planned  between '{}' and '{}'
    )
    select 	app_id,
        --
        planned, --at time zone 'UTC' planned,
        updated, --at time zone 'UTC' updated,
        started, --at time zone 'UTC' started,
        finished, --at time zone 'UTC' finished,
        duration,
        case when waiting > '0'
             then waiting
        else NULL
        end waiting,
        --
        "source",
        state,
        --
        patient_fio,
        patient_birthday,
        phone,
        email,
        sex,
        --
        policy_number,
        --
        --invoice_id,
        transaction_,
        invoice_created,
        case when sum(i_price) = 0
             then null
             else payment_system
        end,
        case when sum(i_price) = 0
            then null
            else sum(i_price)
        end	invoice_total,
        --
        doctor_fio,
        specialty,
        doctor_provider,
        doctor_partner,
        --
        report_comment,
        --
        rating,
        "comment",
        --
        price,
        --
        description,
        --promo.description_tsvector, NE NUJNO
        code_norm,
        patient_partner,
        --
        case when is_chat
            then 'Да'
            else 'Нет'
        end is_chat,
        case when is_video
            then 'Да'
            else 'Нет'
        end is_video,
        case when is_audio
            then 'Да'
            else 'Нет'
        end is_audio,
        case when is_phone
            then 'Да'
            else 'Нет'
        end is_phone,
        --
        case when good
            then 'Да'
            else 'Нет'
        end good,
        initiator
    from temp
    group by app_id,
        --
        planned, updated, started, finished, 
        duration, waiting, "source", state, patient_fio, patient_birthday,
        phone, email, sex, policy_number, transaction_, invoice_created,
        payment_system, doctor_fio, specialty, doctor_provider, doctor_partner,
        --
        report_comment,
        --
        rating, "comment",
        --
        price,
        --
        description, code_norm, patient_partner,
        --
        is_chat, is_video, is_audio, is_phone,
        --
        good, initiator
    order by finished
    """.format(t_start, t_end)
    
    connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
    data = connection.query(sql)
    connection.close()
    
    data['duration'] = data.duration.apply(str)
    data['waiting'] = data.waiting.apply(str)

    data['duration'] = data.duration.apply(lambda x: x.replace('0 days ', ''))
    data['waiting'] = data.waiting.apply(lambda x: x.replace('0 days ', ''))
    
    full_df = pd.concat([full_df, data])

2020-06-01 2020-06-02
************
2020-07-08 13:33:48 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-07-08 13:33:48 creating ssh tunnel to 172.16.100.19 as root...
2020-07-08 13:33:48 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 44197
2020-07-08 13:33:48 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-07-08 13:33:48 @telemed query:
                    with temp as (
				        select 	a.id app_id,
				            --
				            a.planned, --at time zone 'UTC' planned,
				            a.updated, --at time zone 'UTC' updated,
				            a.started, --at time zone 'UTC' started,
				            a.finished, --at time zone 'UTC' finished,
				            a.finished - a.started duration,
				            a.started - a.planned waiting,
				            --
				            a."source",
				    

In [4]:
#full_df

In [10]:
global writer
writer = None

writer = auto_columns_width(os.getcwd() + '/Report' + '.xlsx',
                            full_df, 
                            'June',
                            writer,
                            1.1)


writer.save()

In [46]:
"""
data['duration'] = data.duration.apply(str)
data['waiting'] = data.waiting.apply(str)

data['duration'] = data.duration.apply(lambda x: x.replace('0 days ', ''))
data['waiting'] = data.waiting.apply(lambda x: x.replace('0 days ', ''))
"""

In [8]:
#data['duration'] = data['duration'].dt.floor('D') + pd.to_timedelta(data['duration'])

In [9]:
#data['waiting'] = data['waiting'].dt.floor('D') + pd.to_timedelta(data['waiting'])

In [10]:
#data

In [47]:
data.to_excel('1234.xlsx')